In [250]:
import nltk
import numpy as np
import pandas as pd
from nltk import ngrams
from nltk.probability import FreqDist
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [251]:
# Step 1: Load the data
data = pd.read_csv('google_play_store_apps_reviews.csv')

# Step 2: Split the data
train_data, test_data = train_test_split(data, test_size = 0.2, random_state = 42)

data.head()

package_name                                             review  \
0  com.facebook.katana   privacy at least put some option appear offli...   
1  com.facebook.katana   messenger issues ever since the last update, ...   
2  com.facebook.katana   profile any time my wife or anybody has more ...   
3  com.facebook.katana   the new features suck for those of us who don...   
4  com.facebook.katana   forced reload on uploading pic on replying co...   

   polarity  
0         0  
1         0  
2         0  
3         0  
4         0

In [274]:
# Step 3: Build the n-gram Language Model
def get_ngrams(text, n):
    tokens = nltk.word_tokenize(text)
    return list(ngrams(tokens, n))

def train_ngram(data, n):
    positive_ngrams = []
    negative_ngrams = []

    for index, row in data.iterrows():
        grams = get_ngrams(row['review'], n)
        if row['polarity'] == 1:
            positive_ngrams.extend(grams)
        elif row['polarity'] == 0:
            negative_ngrams.extend(grams)

    positive_freq = FreqDist(positive_ngrams)
    negative_freq = FreqDist(negative_ngrams)

    return positive_freq, negative_freq

# Step 4: Train the Model
n = 2  # Change to the desired n-gram size
positive_freq, negative_freq = train_ngram(train_data, n)
#positive_freq_minus, negative_freq_minus = train_ngram(train_data, n-1)

In [275]:
positive_freq

FreqDist({('!', '!'): 108, ('.', 'i'): 48, ('it', "'s"): 45, ('this', 'app'): 35, ('it', 'is'): 33, (',', 'but'): 27, ('i', 'love'): 25, ('of', 'the'): 25, ('.', 'it'): 21, ('this', 'game'): 21, ...})

In [276]:
# Step 5: test the n-gram
def test_ngram(data, positive_freq, negative_freq, n, alpha):
    results = []
    
    total_positive_count = sum(positive_freq.values())
    total_negative_count = sum(negative_freq.values())
    V = len(set(positive_freq.keys()).union(set(negative_freq.keys())))
    
    for index, row in data.iterrows():
        ngrams = get_ngrams(row['review'], n)
        positive_probability = 0
        negative_probability = 0
        
        alpha = alpha
        for ngram in ngrams:
            positive_probability += (positive_freq.get(ngram, 0) + alpha) / (total_positive_count + alpha*V)
            negative_probability += (negative_freq.get(ngram, 0) + alpha) / (total_negative_count + alpha*V)
            #print(ngram)
            
        if positive_probability > negative_probability:
            label = 1 
        else:
            label = 0
            
        results.append((row['review'],row['polarity'], label))
        
    return results

In [277]:
results = test_ngram(test_data, positive_freq, negative_freq, n , 0)
results

[(" love/hate has bug and security issues. i tried to report that facebook and google plus have security issues and it wouldn't allow me to do so! well i just did didn't i! ......",
  0,
  0),
 (' whatsapp i use this app now that blackberry messenger has basically gone away. my friends & family live all over the world. this really helps keep us in touch!',
  1,
  1),
 (' usefully verry  nice app', 1, 1),
 (' fonts why in the heck is this thing analysing my fonts??? not really quick browsing when i have to wait 5minutes for the fonts to load. are you asking my opinion? avoid this. terrible',
  0,
  0),
 (" app doesn't work after latest upgrade the facebook app refuses to work on my mobile data (3g) after the latest upgrade! it says it cannot connect right now.",
  0,
  0),
 (" wechat wallet still disappears people have been using wechat wallet in the english ios app for ages now with no problems, and yet when i put my android wechat app into english, wallet disappears! please fix the en

In [278]:
# Step 6: Evaluate the model on the test set
from sklearn.metrics import accuracy_score, precision_score, recall_score

actual_labels = [result[1] for result in results]
predicted_labels = [result[2] for result in results]

accuracy = accuracy_score(actual_labels, predicted_labels)
precision = precision_score(actual_labels, predicted_labels)
recall = recall_score(actual_labels, predicted_labels)

print("accuracy =", accuracy)
print("precision =", precision)
print("recall =", recall)


accuracy = 0.7318435754189944
precision = 0.5384615384615384
recall = 0.660377358490566


In [279]:
alphas = np.arange(0, 1.1, 0.1)  # From 0 to 1 with a step of 0.1
accuracies = []

for alpha in alphas:
    results = test_ngram(test_data, positive_freq, negative_freq, n, alpha)
    actual_labels = [result[1] for result in results]
    predicted_labels = [result[2] for result in results]
    
    accuracy = accuracy_score(actual_labels, predicted_labels)
    accuracies.append(accuracy)

plt.plot(alphas, accuracies)
plt.title('Accuracy vs Smoothing Parameter')
plt.xlabel('Alpha')
plt.ylabel('Accuracy')
plt.show()

In [258]:
import re

def custom_tokenizer(text):
    pattern = r'\b\w+\b'

    tokens = re.findall(pattern, text)

    return tokens